ip -details link show can0
# Should list something like: “can0: <NOARP,UP,LOWER_UP> ... state UP ...”

sudo ip link set can0 up type can bitrate 500000 dbitrate 1000000 fd on
# dyno baud rate: 500k

# (Re-)load all CAN drivers
sudo modprobe can && sudo modprobe can_raw       # generic CAN support
sudo modprobe mttcan                             # on-board CAN controller
sudo modprobe kvaser_usb                         # USB Kvaser Leaf

# on computer
sudo modprobe can       # core CAN support
sudo modprobe can_raw   # raw-socket protocol
sudo modprobe can_dev   # network interface “canX” support
sudo modprobe kvaser_usb

# Bring up the on-board CAN as can0 @ 500 kbps
# Bring up the USB Kvaser as can1 @ 500 kbps
# (replace ‘can1’ with whatever `ip link` shows if it’s different)
sudo ip link set can0 down                         # if it was already up
sudo ip link set can0 type can bitrate 500000
sudo ip link set can0 up
sudo ip link set can1 down
sudo ip link set can1 type can bitrate 500000
sudo ip link set can1 up
sudo ip link set can2 down
sudo ip link set can2 type can bitrate 500000
sudo ip link set can2 up

ls /sys/class/net/ | grep can
i2cdetect -l (find what the number i2c device lie and change that parameter)

# Verify both are up
ip link show can0
ip link show can1

# Quick CLI test
candump can0   # should show anything on the on-board bus
candump can1   # likewise for the Leaf

sudo modprobe kvaser_usb   # already done, but harmless to repeat
sudo dmesg -w | grep -i kvaser &



# 1) grab the sources
git clone https://github.com/AD-EYE/socketcan_kvaser_drivers.git
cd socketcan_kvaser_drivers

# 2) make sure headers & build tools are present
sudo apt-get install --yes build-essential linux-headers-$(uname -r)

# 3) (optional but tidy) remove any older copy
sudo make uninstall

# 4) build & install the module
make
sudo make install      # installs into /lib/modules/$(uname -r)/updates
sudo depmod -a 

sudo modprobe kvaser_usb           # load the module
dmesg | tail                       # should show “Kvaser Leaf Light v2 … registered”
lsmod | grep kvaser_usb            # make sure it’s really loaded
ip -details link show | grep -A2 can

In [4]:
import can
bus = can.interface.Bus(channel='can1', interface='socketcan')
for msg in bus:
    print(f"ID: {msg.arbitration_id:03X}  Data: {msg.data.hex()}")

KeyboardInterrupt: 

In [53]:
from canlib import canlib
n = canlib.getNumberOfChannels()
print("Channels:", n)
for ch in range(n):
    info = canlib.ChannelData(ch)
    print(ch, info.channel_name, info.card_serial_no)


Channels: 0


In [47]:
from canlib import canlib, Frame          # Frame only if you want typing help

CHANNEL      = 0
BITRATE      = canlib.canBITRATE_500K     # match your bus
READ_TIMEOUT = 500                       # ms

ch = canlib.openChannel(CHANNEL, canlib.canOPEN_EXCLUSIVE)
ch.setBusParams(BITRATE)
ch.busOn()
print("Listening…  (Ctrl-C to stop)")

try:
    while True:
        try:
            frame = ch.read(timeout=READ_TIMEOUT)
            print(f"ID {frame.id:03X}, DLC {frame.dlc}, Data {frame.data}")
        except canlib.CanNoMsg:          # ← the right exception class
            # nothing arrived within READ_TIMEOUT; just keep looping
            continue
except KeyboardInterrupt:
    pass
finally:
    ch.busOff()
    ch.close()



CanNotFound: Specified device not found (-3)

In [8]:
from canlib import canlib

ch0 = canlib.openChannel(0)   # first virtual channel
ch1 = canlib.openChannel(1)   # second virtual channel
ch0.setBusParams(canlib.canBITRATE_500K)
ch0.busOn()
msg = ch0.read()
print(msg)


CanNotFound: Specified device not found (-3)

In [10]:
import can
import threading

def reader(bus: can.Bus, name: str):
    for msg in bus:
        print(f"[{name}] {msg.timestamp:.6f} ID=0x{msg.arbitration_id:X}  Data={msg.data.hex()}")

def main():
    # create two separate buses for can0 and can1
    bus0 = can.interface.Bus(channel='can0', bustype='socketcan_native')
    bus1 = can.interface.Bus(channel='can1', bustype='socketcan_native')  # :contentReference[oaicite:1]{index=1}

    # spin up background threads to read each
    t0 = threading.Thread(target=reader, args=(bus0, 'CAN0'), daemon=True)
    t1 = threading.Thread(target=reader, args=(bus1, 'CAN1'), daemon=True)
    t0.start()
    t1.start()

    print("Listening on CAN0 and CAN1. Ctrl-C to exit.")
    try:
        while True:
            pass
    except KeyboardInterrupt:
        print("Shutting down.")

if __name__ == "__main__":
    main()




/tmp/ipykernel_3843/3554274743.py:10: DeprecationWarning: The 'bustype' argument is deprecated since python-can v4.2.0, and scheduled for removal in python-can v5.0.0. Use 'interface' instead.
  bus0 = can.interface.Bus(channel='can0', bustype='socketcan_native')


CanInterfaceNotImplementedError: Unknown interface type "socketcan_native"

In [5]:
import time

import can.util
import cantools
import can
# from can.listener import Listener as _Listener
# import can.interfaces
# from can.interface import interface as _interface
# ── STEP 1: Load your DBC file ────────────────────────────────────────────────
# Replace 'path/to/my_dyno.dbc' with the actual path to your DBC
dbc_path = '/home/guiliang/Desktop/DriveRobot/KAVL_V3.dbc'
db = cantools.database.load_file(dbc_path)
msg_def = db.get_message_by_name('Speed_and_Force')

# (Optional) Print out all known message names and IDs from the DBC
print("Loaded messages from DBC:")
for msg in db.messages:
    # msg.frame_id is the numeric CAN ID
    # msg.name is the symbolic name
    print(f"  ID={msg.frame_id:X}  Name={msg.name}  Size={msg.length} bytes")
print("───────────────────────────────────────────────────────────────────────────\n")


# ── STEP 2: Open the SocketCAN interface ('can0') ────────────────────────────
# Note: bus_type='socketcan' is what python-can expects on Linux
#       channel='can0' matches the interface you brought up.
try:
    bus = can.interface.Bus(channel='can0', interface='socketcan')
except OSError as e:
    print("ERROR: Could not open CAN interface 'can0'.\n"
          "Make sure you've run:\n"
          "    sudo ip link set can0 up type can bitrate 500000\n"
          "and that no other process is blocking 'can0'.")
    raise

print("Listening on can0... (press Ctrl+C to exit)")

# ── STEP 3: Loop, receive and decode CAN messages ─────────────────────────────
try:
    while True:
        msg = bus.recv(timeout=1.0)  # Wait up to 1s for a frame
        if msg is None:
            # No frame within timeout: you can continue or do housekeeping
            continue

        can_id = msg.arbitration_id
        print(f"can_id:{can_id}")
        data = msg.data  # a bytes-like of length up to 8 (or 64 if FD)

        # Attempt to decode using the DBC:
        try:
            decoded = db.decode_message(can_id, data)
            force = decoded['Force_N']
        except KeyError:
            # ID not in DBC → ignore or log as “unknown ID”
            # e.g.:
            # print(f"Unknown CAN ID 0x{can_id:X}, raw={data.hex()}")
            continue

        # If decoding succeeds, ‘decoded’ is a dict: {signal_name: value, ...}
        # Example: {'EngineSpeed': 1532.5, 'ThrottlePos': 27.2}
        timestamp = time.time()
        print(f"[{timestamp:.3f}] ID=0x{can_id:X} → {decoded}")

except KeyboardInterrupt:
    print("\nStopped by user. Exiting.")
except Exception as e:
    print("Unexpected error:", e)
finally:
    bus.shutdown()


Loaded messages from DBC:
  ID=1  Name=Front_Axle_Force_Signals  Size=8 bytes
  ID=2  Name=Rear_Axle_Force_Signals  Size=8 bytes
  ID=3  Name=Total_SPD_and_Force  Size=8 bytes
  ID=4  Name=Front_Axle_PWR_and_SPD_Signals  Size=8 bytes
  ID=5  Name=Rear_Axle_PWR_and_SPD_Signals  Size=8 bytes
  ID=6  Name=Total_PWR_and_ACC  Size=8 bytes
  ID=7  Name=Time_and_Misc  Size=8 bytes
  ID=8  Name=Set_Values  Size=8 bytes
  ID=9  Name=Speed_and_Force  Size=8 bytes
  ID=A  Name=Vehicle_cooling_fan  Size=8 bytes
  ID=B  Name=V_ACT  Size=8 bytes
───────────────────────────────────────────────────────────────────────────

Listening on can0... (press Ctrl+C to exit)
can_id:9
[1750556816.523] ID=0x9 → {'Speed_kph': 0.0008015559869818389, 'Force_N': -1.2953749895095825}
can_id:9
[1750556816.524] ID=0x9 → {'Speed_kph': 0.0008024004637263715, 'Force_N': -1.3266314268112183}
can_id:9
[1750556816.524] ID=0x9 → {'Speed_kph': 0.00076377356890589, 'Force_N': -1.4267339706420898}
can_id:9
[1750556816.524] ID=0x

In [7]:
import time
import cantools
import can

# ─── STEP 1: Load the DBC file ────────────────────────────────────────────────
#
# Replace this path with wherever you placed KAVL_V3.dbc on your Jetson filesystem.
DBC_PATH = '/home/guiliang/Desktop/DriveRobot/KAVL_V3.dbc'

try:
    db = cantools.database.load_file(DBC_PATH)
except FileNotFoundError:
    print(f"ERROR: Cannot find DBC at '{DBC_PATH}'. Make sure the path is correct.")
    raise

# (Optional) Verify that "Speed_and_Force" exists in the DBC:
try:
    msg_def = db.get_message_by_name('Speed_and_Force')
except KeyError:
    print("ERROR: The DBC does not contain a message named 'Speed_and_Force'.")
    raise

print(f"Loaded DBC message:\n"
      f"  Name = {msg_def.name}\n"
      f"  ID   = 0x{msg_def.frame_id:03X}\n"
      f"  Signals = {[sig.name for sig in msg_def.signals]}\n")

# ─── STEP 2: Open the SocketCAN interface ─────────────────────────────────────
#
# Make sure you've already run something like:
#   sudo ip link set can0 up type can bitrate 500000 dbitrate 1000000 fd on
#
CAN_INTERFACE = 'can0'

try:
    bus = can.interface.Bus(channel=CAN_INTERFACE, bustype='socketcan')
except OSError as e:
    print(f"ERROR: Could not open CAN interface '{CAN_INTERFACE}'.\n"
          f"Make sure you've run:\n"
          f"    sudo ip link set {CAN_INTERFACE} up type can bitrate 500000 dbitrate 1000000 fd on\n"
          f"and that no other process is blocking '{CAN_INTERFACE}'.")
    raise

print(f"Listening on {CAN_INTERFACE} for 'Speed_and_Force' (ID=0x{msg_def.frame_id:03X})...\n")

# ─── STEP 3: Receive frames & decode Force_N ─────────────────────────────────
#
# The DBC says that "Force_N" occupies bits 24–55 (32 bits, little-endian, float),
# with gain=1.0 and offset=0.0. Cantools will handle the raw‐to‐float for us.

try:
    while True:
        msg = bus.recv(timeout=1.0)  # wait up to 1 second for a frame
        if msg is None:
            # No CAN frame arrived in the last second; just loop again.
            continue

        # Only decode if the incoming ID matches Speed_and_Force (0x009)
        if msg.arbitration_id != msg_def.frame_id:
            continue

        # Decode the raw bytes into a dict of {signal_name: value}
        try:
            decoded_signals = db.decode_message(msg.arbitration_id, msg.data)
        except KeyError:
            # If the ID somehow isn’t in the DBC, skip it
            continue

        # Extract the Force_N signal
        force_value = decoded_signals.get('Force_N', None)
        if force_value is None:
            # For some reason, Force_N wasn’t in the decoded dict
            print(f"[0x{msg.arbitration_id:03X}] Received frame but no 'Force_N' signal.")
            continue

        # Timestamp, ID, and Force_N in Newtons
        ts = time.time()
        print(f"[{ts:.3f}] ID=0x{msg.arbitration_id:03X}  →  Force_N = {force_value:.3f} N")

except KeyboardInterrupt:
    print("\nInterrupted by user. Exiting...")
finally:
    bus.shutdown()


Loaded DBC message:
  Name = Speed_and_Force
  ID   = 0x009
  Signals = ['Speed_kph', 'Force_N']

Listening on can0 for 'Speed_and_Force' (ID=0x009)...



/tmp/ipykernel_4181/2352847903.py:36: DeprecationWarning: The 'bustype' argument is deprecated since python-can v4.2.0, and scheduled for removal in python-can v5.0.0. Use 'interface' instead.
  bus = can.interface.Bus(channel=CAN_INTERFACE, bustype='socketcan')


[1750556964.484] ID=0x009  →  Force_N = 2.557 N
[1750556964.485] ID=0x009  →  Force_N = 2.553 N
[1750556964.486] ID=0x009  →  Force_N = 2.597 N
[1750556964.486] ID=0x009  →  Force_N = 2.672 N
[1750556964.486] ID=0x009  →  Force_N = 2.742 N
[1750556964.486] ID=0x009  →  Force_N = 2.773 N
[1750556964.486] ID=0x009  →  Force_N = 2.750 N
[1750556964.486] ID=0x009  →  Force_N = 2.681 N
[1750556964.486] ID=0x009  →  Force_N = 2.595 N
[1750556964.487] ID=0x009  →  Force_N = 2.525 N
[1750556964.487] ID=0x009  →  Force_N = 2.499 N
[1750556964.487] ID=0x009  →  Force_N = 2.533 N
[1750556964.487] ID=0x009  →  Force_N = 2.625 N
[1750556964.487] ID=0x009  →  Force_N = 2.768 N
[1750556964.487] ID=0x009  →  Force_N = 2.946 N
[1750556964.487] ID=0x009  →  Force_N = 3.138 N
[1750556964.488] ID=0x009  →  Force_N = 3.311 N
[1750556964.488] ID=0x009  →  Force_N = 3.433 N
[1750556964.488] ID=0x009  →  Force_N = 3.488 N
[1750556964.488] ID=0x009  →  Force_N = 3.472 N
[1750556964.489] ID=0x009  →  Force_N = 

In [18]:
import time
import cantools
import can

# ─── STEP 1: Load the DBC file ────────────────────────────────────────────────
#
# Change this path to point at your actual KAVL_V3.dbc file.
DBC_PATH = '/home/guiliang/Desktop/DriveRobot/KAVL_V3.dbc'

try:
    db = cantools.database.load_file(DBC_PATH)
except FileNotFoundError:
    print(f"ERROR: Cannot find DBC at '{DBC_PATH}'.")
    raise

# Grab the message definition for "Speed_and_Force" (ID = 0x009)
try:
    speed_force_msg = db.get_message_by_name('Speed_and_Force')
except KeyError:
    print("ERROR: 'Speed_and_Force' not found in the DBC.")
    raise

CAN_INTERFACE = 'can0'

# This variable will always contain the most recent Force_N reading (in Newtons).
latest_force = None

# ─── STEP 2: Open the SocketCAN interface ─────────────────────────────────────
#
# Make sure you've already run (or something similar) on the Jetson:
#   sudo ip link set can0 up type can bitrate 500000 dbitrate 1000000 fd on
#
try:
    bus = can.interface.Bus(channel=CAN_INTERFACE, bustype='socketcan')
except OSError:
    print(f"ERROR: Cannot open CAN interface '{CAN_INTERFACE}'.")
    print("Make sure you have run something like:\n"
          "    sudo ip link set can0 up type can bitrate 500000 dbitrate 1000000 fd on")
    raise

print(f"Listening on {CAN_INTERFACE} for ID=0x{speed_force_msg.frame_id:03X} ('Speed_and_Force')…")
print("Press Ctrl+C to exit.\n")

# ─── STEP 3: Loop forever, read Force_N, print & store ────────────────────────
try:
    while True:
        # Wait up to 1 second for a CAN frame; msg will be a can.Message or None.
        msg = bus.recv(timeout=1.0)
        if msg is None:
            # No frame in the last second; skip to next iteration.
            continue

        # Only decode if the ID matches 0x009:
        if msg.arbitration_id != speed_force_msg.frame_id:
            continue

        # Decode raw payload into a dict {signal_name: value}
        try:
            decoded = db.decode_message(msg.arbitration_id, msg.data)
        except KeyError:
            # ID wasn’t in the DBC (unlikely here), so skip.
            continue

        # Pull out the Force_N field (in Newtons)
        force_value = decoded.get('Force_N')
        if force_value is None:
            # If for some reason Force_N isn’t present, skip.
            continue

        # Update our “live” variable:
        latest_force = force_value

        # Print live to console (you could apply formatting as needed)
        timestamp = time.time()
        print(f"[{timestamp:.3f}] Force_N = {latest_force:.3f} N")

        # (Optional) If you want to do something with latest_force each loop,
        # you can access the Python variable `latest_force` here.

except KeyboardInterrupt:
    print("\nInterrupted by user. Exiting…")
finally:
    bus.shutdown()


/tmp/ipykernel_11465/3982627924.py:34: DeprecationWarning: The 'bustype' argument is deprecated since python-can v4.2.0, and scheduled for removal in python-can v5.0.0. Use 'interface' instead.
  bus = can.interface.Bus(channel=CAN_INTERFACE, bustype='socketcan')


Listening on can0 for ID=0x009 ('Speed_and_Force')…
Press Ctrl+C to exit.


Interrupted by user. Exiting…


In [17]:
import time
import cantools
import can

# ─── STEP 1: Load the DBC file ────────────────────────────────────────────────
#
# Change this path to point at your actual KAVL_V3.dbc file.
DBC_PATH = '/home/guiliang/Desktop/DriveRobot/KAVL_V3.dbc'

try:
    db = cantools.database.load_file(DBC_PATH)
except FileNotFoundError:
    print(f"ERROR: Cannot find DBC at '{DBC_PATH}'.")
    raise

# Grab the message definition for "Speed_and_Force" (ID = 0x009)
try:
    speed_force_msg = db.get_message_by_name('Speed_and_Force')
except KeyError:
    print("ERROR: 'Speed_and_Force' not found in the DBC.")
    raise

CAN_INTERFACE = 'can0'

# ─── STEP 2: Prepare history container ───────────────────────────────────────
#
# We'll store every Force_N reading in this list in chronological order.
force_history = []  # Empty list at start

# This variable will also hold the most recent Force_N reading.
latest_force = None

# ─── STEP 3: Open the SocketCAN interface ─────────────────────────────────────
#
# Make sure you've already run (or something similar) on the Jetson:
#   sudo ip link set can0 up type can bitrate 500000 dbitrate 1000000 fd on
#
try:
    bus = can.interface.Bus(channel=CAN_INTERFACE, bustype='socketcan')
except OSError:
    print(f"ERROR: Cannot open CAN interface '{CAN_INTERFACE}'.")
    print("Make sure you have run something like:\n"
          "    sudo ip link set can0 up type can bitrate 500000 dbitrate 1000000 fd on")
    raise

print(f"Listening on {CAN_INTERFACE} for ID=0x{speed_force_msg.frame_id:03X} ('Speed_and_Force')…")
print("Press Ctrl+C to exit.\n")

# ─── STEP 4: Loop forever, read Force_N, print & store ────────────────────────
try:
    while True:
        # Wait up to 1 second for a CAN frame; msg will be a can.Message or None.
        msg = bus.recv(timeout=1.0)
        if msg is None:
            # No frame in the last second; skip to next iteration.
            continue

        # Only decode if the ID matches 0x009:
        if msg.arbitration_id != speed_force_msg.frame_id:
            continue

        # Decode raw payload into a dict {signal_name: value}
        try:
            decoded = db.decode_message(msg.arbitration_id, msg.data)
        except KeyError:
            # ID wasn’t in the DBC (unlikely here), so skip.
            continue

        # Pull out the Force_N field (in Newtons)
        force_value = decoded.get('Force_N')
        if force_value is None:
            # If for some reason Force_N isn’t present, skip.
            continue

        # Update our “live” variable:
        latest_force = force_value

        # Append to history list:
        force_history.append(force_value)

        # Print live to console
        timestamp = time.time()
        print(f"[{timestamp:.3f}] Force_N = {latest_force:.3f} N")

        # (Optional) If you want to break out after collecting a certain number:
        # if len(force_history) >= 1000:
        #     break

except KeyboardInterrupt:
    print("\nInterrupted by user. Exiting…")
finally:
    bus.shutdown()

    # ─── STEP 5: (Optional) Do something with force_history ───────────────────
    #
    # At this point, force_history is a list of all recorded Force_N values (floats).
    # You could, for example, print the last few samples, save to disk, etc.
    print(f"\nTotal samples collected: {len(force_history)}")
    print("Last five readings:", force_history[-5:])
    # # To save to a CSV file:
    # import csv
    # with open('force_history.csv', 'w', newline='') as f:
    #     writer = csv.writer(f)
    #     writer.writerow(['timestamp','force_N'])
    #     # If you also stored timestamps alongside force_history, write them here.
    #     for val in force_history:
    #         writer.writerow([<timestamp>, val])


/tmp/ipykernel_11465/3553976443.py:39: DeprecationWarning: The 'bustype' argument is deprecated since python-can v4.2.0, and scheduled for removal in python-can v5.0.0. Use 'interface' instead.
  bus = can.interface.Bus(channel=CAN_INTERFACE, bustype='socketcan')


Listening on can0 for ID=0x009 ('Speed_and_Force')…
Press Ctrl+C to exit.


Interrupted by user. Exiting…

Total samples collected: 0
Last five readings: []


In [16]:
import time
import cantools
import can

# ── STEP 1: Load your DBC file ────────────────────────────────────────────────
dbc_path = '/home/guiliang/Desktop/DriveRobot/KAVL_V3.dbc'
db = cantools.database.load_file(dbc_path)

for msg in db.messages:
    print(f"Name: {msg.name:30s}  ID: 0x{msg.frame_id:03X} ({msg.frame_id})  DLC: {msg.length} bytes")


# Get the Speed_and_Force message definition
msg_def = db.get_message_by_name('Speed_and_Force')
print(msg_def)
target_id = msg_def.frame_id    # should be 0x81 (129 decimal) if your DBC is set up that way

print(f'target_id: {target_id}')
# (Optional) Print out all known message names and IDs from the DBC
print("Loaded messages from DBC:")
for msg in db.messages:
    print(f"  ID=0x{msg.frame_id:X}  Name={msg.name}  Size={msg.length} bytes")
print("───────────────────────────────────────────────────────────────────────────\n")

# ── STEP 2: Open the SocketCAN interface ('can0') ────────────────────────────
try:
    bus = can.interface.Bus(channel='can0', interface='socketcan')
except OSError as e:
    print("ERROR: Could not open CAN interface 'can0'.\n"
          "Make sure you've run:\n"
          "    sudo ip link set can0 up type can bitrate 500000\n"
          "and that no other process is blocking 'can0'.")
    raise

print(f"Listening on can0 for ID=0x{target_id:X} … (press Ctrl+C to exit)")

# ── STEP 3: Loop, receive and decode only ID=0x81 ─────────────────────────────
try:
    while True:
        msg = bus.recv(timeout=1.0)  # Wait up to 1 s for a frame
        if msg is None:
            # No frame within timeout → just loop again
            continue

        # Check if this is the exact ID we care about
        print(f'msg.arbitration_id: {msg.arbitration_id}')
        if msg.arbitration_id == target_id:
            raw_bytes = msg.data    # e.g. b'\x00\x00\x00\x00\x00\x00\x00\x00'
            hex_str   = raw_bytes.hex()
            timestamp = time.time()

            # Decode using the Speed_and_Force definition
            try:
                decoded = msg_def.decode(raw_bytes)
            except Exception as e:
                print(f"[{timestamp:.3f}] Failed to decode ID=0x{target_id:X} raw={hex_str}: {e}")
                continue

            # Extract whichever signals your DBC actually defines. 
            # (Replace 'Speed' and 'Force_N' below with your exact signal names if they differ.)
            speed   = decoded.get('Speed', None)
            force_n = decoded.get('Force_N', None)

            print(
                f"[{timestamp:.3f}] ID=0x{target_id:X}  raw=0x{hex_str.upper()}  →  "
                f"Speed={speed}  Force_N={force_n}"
            )

except KeyboardInterrupt:
    print("\nStopped by user. Exiting.")
except Exception as e:
    print("Unexpected error:", e)
finally:
    bus.shutdown()


SocketcanBus was not properly shut down


Name: Front_Axle_Force_Signals        ID: 0x001 (1)  DLC: 8 bytes
Name: Rear_Axle_Force_Signals         ID: 0x002 (2)  DLC: 8 bytes
Name: Total_SPD_and_Force             ID: 0x003 (3)  DLC: 8 bytes
Name: Front_Axle_PWR_and_SPD_Signals  ID: 0x004 (4)  DLC: 8 bytes
Name: Rear_Axle_PWR_and_SPD_Signals   ID: 0x005 (5)  DLC: 8 bytes
Name: Total_PWR_and_ACC               ID: 0x006 (6)  DLC: 8 bytes
Name: Time_and_Misc                   ID: 0x007 (7)  DLC: 8 bytes
Name: Set_Values                      ID: 0x008 (8)  DLC: 8 bytes
Name: Speed_and_Force                 ID: 0x009 (9)  DLC: 8 bytes
Name: Vehicle_cooling_fan             ID: 0x00A (10)  DLC: 8 bytes
Name: V_ACT                           ID: 0x00B (11)  DLC: 8 bytes
message('Speed_and_Force', 0x9, False, 8, {None: 'Actual speed signal is seen on V_AVT and not on V_ACT_D'})
target_id: 9
Loaded messages from DBC:
  ID=0x1  Name=Front_Axle_Force_Signals  Size=8 bytes
  ID=0x2  Name=Rear_Axle_Force_Signals  Size=8 bytes
  ID=0x3  Name=To

In [1]:
import time
import cantools
import can

# ── STEP 1: Load your DBC file ────────────────────────────────────────────────
dbc_path = '/home/guiliang/Desktop/DriveRobot/KAVL_V3.dbc'
db = cantools.database.load_file(dbc_path)

# Get the Speed_and_Force message definition
msg_def = db.get_message_by_name('Speed_and_Force')
target_id = msg_def.frame_id    # should be 0x81 (129 decimal) if your DBC is set up that way


# 1) Load the database
db = cantools.database.load_file(dbc_path)

# 2) See all messages
for m in db.messages:
    print(m.name, hex(m.frame_id), m.length)

# 3) Grab a Message object by name or ID
msg_def = db.get_message_by_name('Speed_and_Force')
# (or) msg_def = db.get_message_by_frame_id(0x81)

# 4) Inspect that message’s signals
for s in msg_def.signals:
    print(s.name, s.start, s.length, s.scale, s.offset)

# 5) Decode incoming raw bytes into physical values
raw_bytes = b'\x00\x00\x00\x00\x00\x00\x00\x00'
decoded = msg_def.decode(raw_bytes)
# Now decoded is e.g. {'Speed': 0.0, 'Force_N': 0.0, …}

# 6) Encode physical values back into raw bytes
payload = msg_def.encode({'Speed': 42.5, 'Force_N': 1500.0})
# payload is a bytes object ready to send on CAN

# 7) Or let db do ID lookup for you:
decoded_all = db.decode_message(0x81, raw_bytes)
# picks the right Message internally and decodes


Front_Axle_Force_Signals 0x1 8
Rear_Axle_Force_Signals 0x2 8
Total_SPD_and_Force 0x3 8
Front_Axle_PWR_and_SPD_Signals 0x4 8
Rear_Axle_PWR_and_SPD_Signals 0x5 8
Total_PWR_and_ACC 0x6 8
Time_and_Misc 0x7 8
Set_Values 0x8 8
Speed_and_Force 0x9 8
Vehicle_cooling_fan 0xa 8
V_ACT 0xb 8
Speed_kph 0 32 1 0
Force_N 32 32 1 0


EncodeError: The signal "Speed_kph" is required for encoding.